## Follow up with Azure OpenAI the course Building Systems with the ChatGPT API

### L9: Evaluation part II

Evaluate LLM responses where there isn't a single "right answer."

This notebook is based on the course: [Evaluation part II](https://learn.deeplearning.ai/chatgpt-building-system/lesson/10/evaluation-part-ii)

In [1]:
import os
import openai # this will require pip install openai
import tiktoken # this will require pip install tiktoken
import utils #Review the file utils.py

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('AZURE_OPENAI_KEY')
openai.api_type = "azure"
openai.api_version = "2023-08-01-preview"
openai.api_base = "https://cog-moww6huavlklg.openai.azure.com/"


### Run through the end-to-end system to answer the user query

These helper functions are running the chain of promopts that you saw in the earlier videos.

In [2]:
customer_msg = f"""
Háblame del teléfono smartx pro y de la cámara fotosnap, la dslr.
Además, ¿qué televisores o productos relacionados con la televisión tienes?"""

products_by_category = utils.get_products_from_query(customer_msg)
category_and_product_list = utils.read_string_to_list(products_by_category)
product_info = utils.get_mentioned_product_info(category_and_product_list)
assistant_answer = utils.answer_user_msg(user_msg=customer_msg,
                                                   product_info=product_info)

In [3]:
print(products_by_category) 
print(category_and_product_list) 
print(product_info) 

[
    {
        'category': 'Smartphones and Accessories',
        'products': ['SmartX ProPhone']
    },
    {
        'category': 'Cameras and Camcorders',
        'products': ['FotoSnap DSLR Camera']
    },
    {
        'category': 'Televisions and Home Theater Systems',
        'products': ['CineView 4K TV', 'CineView 8K TV', 'CineView OLED TV', 'SoundMax Home Theater', 'SoundMax Soundbar']
    }
]
[{'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']}, {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']}, {'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'CineView 8K TV', 'CineView OLED TV', 'SoundMax Home Theater', 'SoundMax Soundbar']}]
[{'name': 'SmartX ProPhone', 'category': 'Smartphones and Accessories', 'brand': 'SmartX', 'model_number': 'SX-PP10', 'warranty': '1 year', 'rating': 4.6, 'features': ['6.1-inch display', '128GB storage', '12MP dual camera', '5G'], 'description': 'A powerful smartph

In [4]:
print(assistant_answer) 

¡Hola! Con gusto te puedo hablar sobre los productos que mencionaste. El teléfono SmartX Pro es un smartphone potente con una pantalla de 6.1 pulgadas, almacenamiento de 128GB, cámara dual de 12MP y conectividad 5G. La cámara FotoSnap DSLR es una cámara versátil con un sensor de 24.2MP, grabación de video 1080p, pantalla LCD de 3 pulgadas y lentes intercambiables.

En cuanto a televisores, tenemos varios modelos de la marca CineView, incluyendo el modelo 4K de 55 pulgadas, el modelo 8K de 65 pulgadas y el modelo OLED de 55 pulgadas. Todos ellos cuentan con resolución 4K o 8K, HDR y funciones inteligentes. También ofrecemos sistemas de cine en casa de la marca SoundMax, como el modelo Home Theater de 5.1 canales y el modelo Soundbar de 2.1 canales.

¿Te gustaría saber más sobre alguno de estos productos o tienes alguna otra pregunta relacionada con electrónica? Estoy aquí para ayudarte.


### Evaluate the LLM's answer to the user with a rubric, based on the extracted product information

In [7]:
cust_prod_info = {
    'customer_msg': customer_msg,
    'context': product_info
}
print(cust_prod_info)

{'customer_msg': '\nHáblame del teléfono smartx pro y de la cámara fotosnap, la dslr.\nAdemás, ¿qué televisores o productos relacionados con la televisión tienes?', 'context': [{'name': 'SmartX ProPhone', 'category': 'Smartphones and Accessories', 'brand': 'SmartX', 'model_number': 'SX-PP10', 'warranty': '1 year', 'rating': 4.6, 'features': ['6.1-inch display', '128GB storage', '12MP dual camera', '5G'], 'description': 'A powerful smartphone with advanced camera features.', 'price': 899.99}, {'name': 'FotoSnap DSLR Camera', 'category': 'Cameras and Camcorders', 'brand': 'FotoSnap', 'model_number': 'FS-DSLR200', 'warranty': '1 year', 'rating': 4.7, 'features': ['24.2MP sensor', '1080p video', '3-inch LCD', 'Interchangeable lenses'], 'description': 'Capture stunning photos and videos with this versatile DSLR camera.', 'price': 599.99}, {'name': 'CineView 4K TV', 'category': 'Televisions and Home Theater Systems', 'brand': 'CineView', 'model_number': 'CV-4K55', 'warranty': '2 years', 'rat

In [8]:
def eval_with_rubric(test_set, assistant_answer):

    cust_msg = test_set['customer_msg']
    context = test_set['context']
    completion = assistant_answer
    
    system_message = """\
    You are an assistant that evaluates how well the customer service agent \
    answers a user question by looking at the context that the customer service \
    agent is using to generate its response. It is necessary to answer in Spanish
    """

    user_message = f"""\
You are evaluating a submitted answer to a question based on the context \
that the agent uses to answer the question.
Here is the data:
    [BEGIN DATA]
    ************
    [Question]: {cust_msg}
    ************
    [Context]: {context}
    ************
    [Submission]: {completion}
    ************
    [END DATA]

Compare the factual content of the submitted answer with the context. \
Ignore any differences in style, grammar, or punctuation.
Answer the following questions:
    - Is the Assistant response based only on the context provided? (Y or N)
    - Does the answer include information that is not provided in the context? (Y or N)
    - Is there any disagreement between the response and the context? (Y or N)
    - Count how many questions the user asked. (output a number)
    - For each question that the user asked, is there a corresponding answer to it?
      Question 1: (Y or N)
      Question 2: (Y or N)
      ...
      Question N: (Y or N)
    - Of the number of questions asked, how many of these questions were addressed by the answer? (output a number)
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = utils.get_completion_from_messages(messages)
    return response

In [9]:
evaluation_output = eval_with_rubric(cust_prod_info, assistant_answer)
print(evaluation_output)

- Is the Assistant response based only on the context provided? (Y or N)
Y

- Does the answer include information that is not provided in the context? (Y or N)
N

- Is there any disagreement between the response and the context? (Y or N)
N

- Count how many questions the user asked. (output a number)
1

- For each question that the user asked, is there a corresponding answer to it?
Question 1: Y

- Of the number of questions asked, how many of these questions were addressed by the answer? (output a number)
1


### Evaluate the LLM's answer to the user based on an "ideal" / "expert" (human generated) answer.

In [10]:
test_set_ideal = {
    'customer_msg': """\
háblame del teléfono smartx pro y de la cámara fotosnap, la dslr.
Además, ¿qué televisores o productos relacionados con la televisión tienes?""",
    'ideal_answer':"""\
Por supuesto.  El SmartX ProPhone es un potente \
smartphone con funciones avanzadas de cámara. \
Por ejemplo, tiene una cámara dual de 12MP \
Otras características son la conexión inalámbrica 5G y el almacenamiento de 128 GB. \
También tiene una pantalla de 6,1 pulgadas.  Su precio es de 899,99 dólares.

La cámara DSLR FotoSnap es ideal para \
capturar fotos y vídeos impresionantes. \
Algunas funciones incluyen vídeo 1080p, \
LCD de 3 pulgadas, un sensor de 24,2 MP, \
y objetivos intercambiables. \
El precio es de 599,99.

Para televisores y productos relacionados, ofrecemos 3 televisores \


Todos los televisores ofrecen HDR y Smart TV.

El televisor CineView 4K tiene colores vibrantes y funciones inteligentes. \
Algunas de estas características incluyen una pantalla de 55 pulgadas, \
con resolución 4K. Su precio es de 599

El CineView 8K TV es un impresionante televisor 8K. \
Algunas de sus características incluyen una pantalla de 65 pulgadas y \
resolución 8K.  Tiene un precio de 2999,99 €.

El CineView OLED TV te permite experimentar colores vibrantes. \
Algunas características incluyen una pantalla de 55 pulgadas y resolución 4K. \
Su precio es de 1499,99.

También ofrecemos 2 productos de cine en casa, ambos incluyen bluetooth.\
El SoundMax Home Theater es un potente sistema de cine en casa para \
una experiencia de audio inmersiva.
Sus características incluyen 5.1 canales, 1000W de salida y subwoofer inalámbrico.
Su precio es de 399,99 euros.

SoundMax Soundbar es una barra de sonido elegante y potente.
Incluye 2.1 canales, 300 W de potencia y subwoofer inalámbrico.
Su precio es de 199,99

¿Hay alguna pregunta adicional que pueda tener acerca de estos productos \
que usted ha mencionado aquí?
¿O tiene alguna otra pregunta en la que pueda ayudarle?
    """
}

### Check if the LLM's response agrees with or disagrees with the expert answer

This evaluation prompt is from the [OpenAI evals](https://github.com/openai/evals/blob/main/evals/registry/modelgraded/fact.yaml) project.

[BLEU score](https://en.wikipedia.org/wiki/BLEU): another way to evaluate whether two pieces of text are similar or not.

In [11]:
def eval_vs_ideal(test_set, assistant_answer):

    cust_msg = test_set['customer_msg']
    ideal = test_set['ideal_answer']
    completion = assistant_answer
    
    system_message = """\
    You are an assistant that evaluates how well the customer service agent \
    answers a user question by comparing the response to the ideal (expert) response
    Output a single letter and nothing else. 
    """

    user_message = f"""\
You are comparing a submitted answer to an expert answer on a given question. Here is the data:
    [BEGIN DATA]
    ************
    [Question]: {cust_msg}
    ************
    [Expert]: {ideal}
    ************
    [Submission]: {completion}
    ************
    [END DATA]

Compare the factual content of the submitted answer with the expert answer. Ignore any differences in style, grammar, or punctuation.
    The submitted answer may either be a subset or superset of the expert answer, or it may conflict with it. Determine which case applies. Answer the question by selecting one of the following options:
    (A) The submitted answer is a subset of the expert answer and is fully consistent with it.
    (B) The submitted answer is a superset of the expert answer and is fully consistent with it.
    (C) The submitted answer contains all the same details as the expert answer.
    (D) There is a disagreement between the submitted answer and the expert answer.
    (E) The answers differ, but these differences don't matter from the perspective of factuality.
  choice_strings: ABCDE
"""

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = utils.get_completion_from_messages(messages)
    return response

In [12]:
print(assistant_answer)

¡Hola! Con gusto te puedo hablar sobre los productos que mencionaste. El teléfono SmartX Pro es un smartphone potente con una pantalla de 6.1 pulgadas, almacenamiento de 128GB, cámara dual de 12MP y conectividad 5G. La cámara FotoSnap DSLR es una cámara versátil con un sensor de 24.2MP, grabación de video 1080p, pantalla LCD de 3 pulgadas y lentes intercambiables.

En cuanto a televisores, tenemos varios modelos de la marca CineView, incluyendo el modelo 4K de 55 pulgadas, el modelo 8K de 65 pulgadas y el modelo OLED de 55 pulgadas. Todos ellos cuentan con resolución 4K o 8K, HDR y funciones inteligentes. También ofrecemos sistemas de cine en casa de la marca SoundMax, como el modelo Home Theater de 5.1 canales y el modelo Soundbar de 2.1 canales.

¿Te gustaría saber más sobre alguno de estos productos o tienes alguna otra pregunta relacionada con electrónica? Estoy aquí para ayudarte.


In [13]:
eval_vs_ideal(test_set_ideal, assistant_answer)

'A'

In [16]:
assistant_answer_2 = "life is like a box of chocolates"

In [17]:
eval_vs_ideal(test_set_ideal, assistant_answer_2)

'D'